# Introduction to convolutional networks
### Based on Chapter 5 from "Deep Learning with Python" by F. Chollet

In [ ]:
from keras import layers, models
from keras.datasets import mnist
from keras.utils import to_categorical

# MNIST example to get started
- Load data (import bundled with Keras)
- Reshape images and scale 8-bit pixel values between zero and one
- Categorically encode labels, for example: "2" -> [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
# Load data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

def prepare_images(images):
    return images.reshape((-1, 28, 28, 1)).astype('float32') / 255.0

# Reshape images and scale between zero and one
train_images = prepare_images(train_images)
test_images = prepare_images(test_images)

# Categorical encoding of class labels
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

### Define convolutional network and add dense network on top as classifier. 
In the convolutional network, patterns are learned from windows of size 3x3. First convolution layer outputs feature maps of size (26, 26, 32). In the first two dimensions, size has decreased from 28 to 26 because of border effects: without padding, there are only 26 pixel positions where 3x3 windows can be placed along an axis with 28 pixels. The size of the output feature map in the third dimension, set to 32 in the first convolution layer and 64 in the following two, is a configurable parameter corresponding to different "filters", analogous to color channels in RGB image.

_MaxPooling_ layers are used to downsample the feature maps. Max pooling selects the maximum value from windows of size 2x2. Downsampling not only reduces the number of trainable coefficients but also spreads information: the convolution layer following a MaxPooling layer works on 3x3 windows containing information from larger area of the original image. 

In [ ]:
model = models.Sequential()

# Convolutional network

# In the first layers, learn patterns from 3x3 windows
model.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1)))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))

# Classifier
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(10, activation = 'softmax'))

model.compile(optimizer = 'rmsprop',
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

model.summary()

### Train network and evaluate performance on test-set
This step contains no cross-validation and basically only ensures that things are properly set-up. Accuracy on the test set exceeds 99%.

In [ ]:
history = model.fit(train_images, train_labels, epochs = 5, batch_size = 64)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy %.4f' % test_acc)

### Visualize some of the failed predictions

In [ ]:
predicted_labels = to_categorical(model.predict_classes(test_images))

# Indices of failed predictions, there might be simpler way to do this
import numpy as np
failures = np.nonzero(np.sum(np.abs(predicted_labels - test_labels), axis = 1))[0]

import matplotlib.pyplot as plt
%matplotlib inline

def plot_for_ind(ind):
    img_failed = test_images[ind_failed]

    predicted_label = np.nonzero(predicted_labels[ind_failed])[0][0]
    labeled = np.nonzero(test_labels[ind_failed])[0][0]

    plt.imshow(np.reshape(img_failed, (28, 28)), cmap='Greys',  interpolation='nearest')

    plt.title('Predicted %d, labeled %d' % (predicted_label, labeled))
    plt.show()

# Visualize five of failed predictions
for ind_failed in failures[0:5]:
    plot_for_ind(ind_failed)

# Cats vs. Dogs
- Requires loading data from [Kaggle dogs vs. cats](https://www.kaggle.com/c/dogs-vs-cats/data) and moving to `./datasets/dogs-vs-cats-original/`
- Script below splits fraction of data to training, validation and test sets, with dogs and cats in different folders

In [ ]:
import os, shutil

original_data_set_dir = './datasets/dogs-vs-cats-original'
target_base_dir = './datasets/dogs-vs-cats'

def ensure_dir(directory):
    print("Creating %s if not exist" % directory)
    os.makedirs(directory, exist_ok = True)

ensure_dir(target_base_dir)

train_dir = os.path.join(target_base_dir, 'train')
validation_dir = os.path.join(target_base_dir, 'validation')
test_dir = os.path.join(target_base_dir, 'test')

for directory in [train_dir, validation_dir, test_dir]:
    ensure_dir(directory)

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

def copy_files(filenames, src_dir, dst_dir):
    ensure_dir(dst_dir)
    print('Copying %d files from %s to %s' % (len(filenames), src_dir, dst_dir))
    for fname in filenames:
        shutil.copyfile(os.path.join(src_dir, fname), os.path.join(dst_dir, fname))
        
training_ids = [i for i in range(1000)]
validation_ids = [i for i in range(1000, 1500)]
test_ids = [i for i in range(1500, 2000)]

copy_files(filenames = ['cat.{}.jpg'.format(i) for i in training_ids], 
           src_dir = original_data_set_dir, 
           dst_dir = train_cats_dir)

copy_files(filenames = ['dog.{}.jpg'.format(i) for i in training_ids], 
           src_dir = original_data_set_dir, 
           dst_dir = train_dogs_dir)

copy_files(filenames = ['cat.{}.jpg'.format(i) for i in validation_ids], 
           src_dir = original_data_set_dir, 
           dst_dir = validation_cats_dir)

copy_files(filenames = ['dog.{}.jpg'.format(i) for i in validation_ids], 
           src_dir = original_data_set_dir, 
           dst_dir = validation_dogs_dir)

copy_files(filenames = ['cat.{}.jpg'.format(i) for i in test_ids], 
           src_dir = original_data_set_dir, 
           dst_dir = test_cats_dir)

copy_files(filenames = ['dog.{}.jpg'.format(i) for i in test_ids], 
           src_dir = original_data_set_dir, 
           dst_dir = test_dogs_dir)


### Define [ImageDataGenerator](https://keras.io/preprocessing/image/#imagedatagenerator-class) for reading images to training

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

gen_batch_size = 20

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = gen_batch_size,
    class_mode = 'binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size = (150, 150),
    batch_size = gen_batch_size,
    class_mode = 'binary')

# Demonstrate the generator
import matplotlib.pyplot as plt

loop_counter = 0
for data_batch, labels_batch in train_generator:
    print('Data batch shape:', data_batch.shape)
    print('Label batch shape:', labels_batch.shape)
    plt.imshow(data_batch[0])
    plt.title('Dog' if int(labels_batch[0]) == 1 else 'Cat')
    plt.show()
    loop_counter += 1
    if loop_counter > 4:
        break

### Define convolution network
Network consists of four convolution layers (each with max pooling) and a final classification layer.

In [ ]:
from keras import models
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = models.Sequential()
model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D((2, 2)))

# Classifier
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

from keras.optimizers import RMSprop
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(lr = 1e-4), metrics = ['acc'])

model.summary()

### Train the model
Note that generator does not know how many batches makes one epoch, so one needs to define `steps_per_epoch`

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = len(training_ids) / gen_batch_size,
    epochs = 30,
    validation_data = validation_generator,
    validation_steps = len(validation_ids) / gen_batch_size)

model.save('cats_and_dogs_small_1.h5')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'rs-', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()